In [ ]:
import tifffile
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
import torch
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from matplotlib import pyplot as plt
from torch import Tensor

import tifffile

from pathlib import Path
import random
from datetime import datetime
from time import sleep
from tqdm import tqdm

# from customdatasets import SegmentationData
from model import VPTREnc, VPTRDec, VPTRDisc, init_weights
from model import GDL, MSELoss, L1Loss, GANLoss
from utils import get_data
from utils import VidCenterCrop, VidPad, VidResize, VidNormalize, VidReNormalize, VidCrop, VidRandomHorizontalFlip, VidRandomVerticalFlip, VidToTensor
from utils import visualize_batch_clips, save_ckpt, load_ckpt, set_seed, AverageMeters, init_loss_dict, update_summary, write_summary, resume_training
from utils import set_seed
import torch.nn.functional as F


from pathlib import Path
import random

from model import VPTREnc, VPTRDec, VPTRDisc, init_weights, VPTRFormerNAR, VPTRFormerFAR
from model import GDL, MSELoss, L1Loss, GANLoss, BiPatchNCE
from utils import KTHDataset, BAIRDataset, MovingMNISTDataset
from utils import get_dataloader, get_data

import logging

In [ ]:
set_seed(2021)
ckpt_save_dir = Path(working_dir+'trained_transformer')
tensorboard_save_dir = Path(working_dir+'tensorboard/')
resume_AE_ckpt = Path(working_dir+'trained_ae').joinpath('epoch_25.tar')
#resume_ckpt = ckpt_save_dir.joinpath('epoch_179.tar')
resume_ckpt = None

#############Set the logger#########
if not Path(ckpt_save_dir).exists():
        Path(ckpt_save_dir).mkdir(parents=True, exist_ok=True)
logging.basicConfig(level=logging.INFO,
                datefmt='%a, %d %b %Y %H:%M:%S',
                format='%(asctime)s - %(message)s',
                filename=ckpt_save_dir.joinpath('train_log.log').absolute().as_posix(),
                filemode='a')

start_epoch = 0
summary_writer = SummaryWriter(tensorboard_save_dir.absolute().as_posix())
num_past_frames = 15
num_future_frames = 5
encH, encW, encC = 16, 16, 528
img_channels = 1 #3 channels for BAIR
epochs = 3
N = 1
#AE_lr = 2e-4
Transformer_lr = 1e-4
max_grad_norm = 1.0 
rpe = False
lam_gan = 0.001
dropout = 0.1
device = torch.device('cuda:0')
val_per_epochs = 4

#####################Init Dataset ###########################
data_set_name = 'CSD'
dataset_dir = working_dir+'MCS/normalized'
test_past_frames = 10
test_future_frames = 10
# train_loader, val_loader, test_loader, renorm_transform = get_data(N, dataset_dir)#, num_frames = 100, video_range = 1000)
train_loader, val_loader, test_loader, renorm_transform = get_data(N, dataset_dir, num_frames = 20, video_limit = None)

#####################Init model###########################
VPTR_Enc = VPTREnc(img_channels, feat_dim = encC, n_downsampling = 3).to(device)
VPTR_Dec = VPTRDec(img_channels, feat_dim = encC, n_downsampling = 3, out_layer = 'Sigmoid').to(device) #Tanh for KTH and BAIR
VPTR_Enc = VPTR_Enc.eval()
VPTR_Dec = VPTR_Dec.eval()

VPTR_Disc = None
#VPTR_Disc = VPTRDisc(img_channels, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d).to(device)
#VPTR_Disc = VPTR_Disc.eval()
#init_weights(VPTR_Disc)
init_weights(VPTR_Enc)
init_weights(VPTR_Dec)

VPTR_Transformer = VPTRFormerFAR(num_past_frames, num_future_frames, encH=encH, encW = encW, d_model=encC, 
                            nhead=8, num_encoder_layers=12, dropout=dropout, 
                            window_size=4, Spatial_FFN_hidden_ratio=4, rpe=rpe).to(device)

optimizer_D = None
#optimizer_D = torch.optim.Adam(params = VPTR_Disc.parameters(), lr = Transformer_lr, betas = (0.5, 0.999))
optimizer_T = torch.optim.AdamW(params = VPTR_Transformer.parameters(), lr = Transformer_lr)

Transformer_parameters = sum(p.numel() for p in VPTR_Transformer.parameters() if p.requires_grad)
print(f"FAR Transformer num_parameters: {Transformer_parameters}")

#####################Init loss function###########################
loss_name_list = ['T_MSE', 'T_GDL', 'T_gan', 'T_total', 'Dtotal', 'Dfake', 'Dreal']
#gan_loss = GANLoss('vanilla', target_real_label=1.0, target_fake_label=0.0).to(device)
loss_dict = init_loss_dict(loss_name_list)
mse_loss = MSELoss()
gdl_loss = GDL(alpha = 1)

#load the trained autoencoder, we initialize the discriminator from scratch, for a balanced training
loss_dict, start_epoch = resume_training({'VPTR_Enc': VPTR_Enc, 'VPTR_Dec': VPTR_Dec}, {}, resume_AE_ckpt, loss_name_list)

if resume_ckpt is not None:
    loss_dict, start_epoch = resume_training({'VPTR_Transformer': VPTR_Transformer}, 
                                            {'optimizer_T':optimizer_T}, resume_ckpt, loss_name_list)

In [ ]:
vid = tifffile.memmap(working_dir + 'MCS/normalized/200803/01_baseline_norm.tif', mode='r+')
x = vid[:100]

# # flatten the array into a 1D array
# flat_arr = x.flatten()

# # calculate the mean and standard deviation of the flattened array
# mean = np.mean(flat_arr)
# std = np.std(flat_arr)

# print("Mean:", mean)
# print("Standard deviation:", std)
x = torch.from_numpy(x).unsqueeze(1).unsqueeze(0)
x = F.interpolate(x, size=(1, 128, 128), mode='nearest')
# for i in range(T):
#   x[i, ...] = transforms.Normalize(1.0937392, 0.11474588)(x[i, ...])
x = transforms.Normalize(1.0937392, 0.11474588)(x)
plt.imshow(x[0, 0, 0])  # Show the first frame of the normalized tensor

x = x.to(device)
print(x.shape)



In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
# print(rec_frames.shape)
x_frame = x[0][50].squeeze().cpu()
with torch.no_grad():
  rec_frames = VPTR_Dec(VPTR_Enc(x))
  pred_frame = rec_frames[0][50].squeeze().cpu()
  p_r = F.interpolate(rec_frames, size=(1, 256, 256), mode='nearest')

  axs[0].imshow(x_frame)
  axs[1].imshow(pred_frame)